In [1]:
# 基础的asm文件，用于构建CFG
fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/inputs/small/small.asm"

# 带bss段和data段的asm文件，用于查看全局变量地址空间
seg_fp = r'/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/inputs/small/smallD.asm'

# 带源码的asm文件，用于源码和汇编代码对照
code_fp = r"/Users/gugujixiao/workspace/project/HWMemory/Code/static analysis/hw-memory/inputs/small/smallL.asm"


In [2]:
'''读指令, 指令对象化'''
from cfg.cfg import proc_identify
from cfg.isa import Instruction, AddrMode
from typing import Tuple
from cfg.read_asm import AsmFileReader, StatementType
from cache_analysis.read_segment import segmentReader
from cfg.read_codeMix import AsmCodeReader, MixStatementType

reader =AsmFileReader(fp)
segreader = segmentReader(seg_fp)
codereader = AsmCodeReader(code_fp)

statements = list()
code_stat = list()

for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

for i in range(len(codereader.statements)):
    s: Tuple[MixStatementType, tuple]
    if codereader.statements[i][0] == MixStatementType.codeline:
        codeline = codereader.statements[i][1]
        addr = ""
        not_find = True
        k = i
        while not_find:
            k += 1
            if codereader.statements[k][0] == MixStatementType.asm:
                addr =  int(codereader.statements[k][1][0],16) 
                not_find = False

        code_stat.append((addr,codeline,"loopname"))

procs = proc_identify(statements)
# for proc in procs:
#     print(proc.name, proc.beg_addr.hex_str())
#     if False and proc.name == 'main':
#         for inst in proc.instruction:
#             print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
# print()

In [3]:
'''分proc'''
from graphviz import Digraph
from cfg.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

'output/procedures.svg'

In [4]:
'''处理call_graph'''
from cfg.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

'output/call_graph.svg'

In [5]:
'''生成TCFG和loop信息'''
from cfg.cfg import TCfg


tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()
# tcfg.add_loop_bound(r'D:\workspace\hw-memory\benchmarks\loop_bound.txt')
# tcfg.add_loop_bound(r'D:\workspace\Gitdocuments\hw-memory\benchmarks\loop_bound.txt')

loop_range =list()
with open("./output/cfg_addr.txt","w") as f:
    for l in tcfg.loops:
        #f.write(l.name+"\n")
        #print(l.name)
        for n in l.all_nodes:
            ins = n.instructions
            #head = ins[0].addr
            head,end = n.inst_range
            string = l.name[1:]+" "+head.hex_str()+"-"+end.hex_str()+"\n"
            loop_range.append(([head.val(),end.val()],l.name))
            f.write(string)
        f.write("\n")


loop_code_range =list()


for i in code_stat:
    for l in loop_range:
        if l[0][0]<i[0]<l[0][1]:
            loop_code_range.append((i[1],l[1]))

for i in loop_code_range:
    print(i)


for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
     temp = n.inside_loop.name if n.inside_loop is not None else 'None'
     print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




('/home/guguji/big/main.c:159', 'l74')
('/home/guguji/big/main.c:159', 'l62')
('/home/guguji/big/main.c:160', 'l74')
('/home/guguji/big/main.c:160', 'l62')
('/home/guguji/big/main.c:176', 'l61')
('/home/guguji/big/main.c:176', 'l50')
('/home/guguji/big/main.c:177', 'l61')
('/home/guguji/big/main.c:177', 'l50')
('/home/guguji/big/main.c:218 (discriminator 3)', 'l2')
('/home/guguji/big/main.c:218 (discriminator 3)', 'l1')
('/home/guguji/big/main.c:218 (discriminator 3)', 'l0')
('/home/guguji/big/main.c:215 (discriminator 3)', 'l2')
('/home/guguji/big/main.c:215 (discriminator 3)', 'l1')
('/home/guguji/big/main.c:215 (discriminator 3)', 'l0')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l6')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l19')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l38')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l3')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l13')
('/home/guguji/big/main.c:235 (discriminator 3)', 'l29')
('/home/gugu

In [6]:
from rw_analysis.rw_package import loadstore_Obj
lds_obj = loadstore_Obj(segreader,tcfg)

#用于给后面解决兼容性问题的数据
lsproc = lds_obj.lsproc
ls_loop_info = lds_obj.loop_info
lds_table = lds_obj.lsproc.ls_table

#for i in lds_table:
#    if i.target_num == 1:
#        print("single",i.ins.tokens,i.reg_target,i.final_addr,i.addr_offset,"是否找到",i.is_find,i.node.name) 
#    else:
#        print("muti",i.ins.tokens,i.reg_target_list[0],i.reg_target_list[1],i.addr_offset,i.final_addr,"是否找到",i.is_find,i.node.name)


#for k,v in ls_loop_info.items():
#    print(k,v)

l2 {1056: 5.6923076923076925, 1057: 5.6923076923076925, 1058: 5.6923076923076925, 1059: 5.6923076923076925, 1060: 5.6923076923076925, 1061: 5.6923076923076925, 1062: 5.6923076923076925, 1063: 5.6923076923076925, 1064: 5.6923076923076925, 1065: 5.6923076923076925, 1066: 5.6923076923076925, 1067: 5.6923076923076925, 1068: 5.6923076923076925, 1069: 5.6923076923076925, 1070: 5.6923076923076925, 1071: 5.6923076923076925, 1072: 5.6923076923076925, 1073: 5.6923076923076925, 1074: 5.6923076923076925, 1075: 5.6923076923076925, 1076: 5.6923076923076925, 1077: 5.6923076923076925, 1078: 5.6923076923076925, 1079: 5.6923076923076925, 1080: 5.6923076923076925, 1081: 5.6923076923076925, 1082: 5.6923076923076925, 1083: 5.6923076923076925, 1084: 5.6923076923076925, 1085: 5.6923076923076925, 1086: 5.6923076923076925, 1087: 5.6923076923076925, 1088: 5.6923076923076925, 1089: 5.6923076923076925, 1090: 5.6923076923076925, 1091: 5.6923076923076925, 1092: 5.6923076923076925, 1093: 5.6923076923076925, 1094: 5.

In [7]:
# from newCFG.heat_analysis import loop_heat
# test=loop_heat(tcfg,lsproc,r'C:\Users\13377\Desktop\hw-memory-master (5)\hw-memory-master\benchmarks\final_benchmark\spec_benchD.asm')
# test.do_it()


In [8]:

# TODO.
from cache_analysis.cache_risk_level import CacheRisk

cache_test = CacheRisk(tcfg, lsproc, r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm',
                       r'D:\workspace\hw-memory\cache_analysis\new_cache\input\cache_information.in')
cache_test.test()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\workspace\\hw-memory\\benchmarks\\final_benchmark\\spec_benchmarkD.asm'

In [ ]:

# from newCFG.rw_condition import RWProc, RWType
# from newCFG.rw_condition_out import RWOut_Proc



# rwout = RWOut_Proc(tcfg_nodes,segreader,rwproc.rw_table,tcfg.loops)




